In [2]:
import os
import json
import requests
import sqlite3
import pandas as pd

def fetch_data(url, season, round, data_type):
    """Fetch data from the API, use caching to save and retrieve the data locally."""
    directory = f'cache/{season}'
    filename = f'{directory}/{data_type}_{round}.json'
    if not os.path.exists(directory):
        os.makedirs(directory)

    if os.path.isfile(filename):
        with open(filename, 'r') as file:
            return json.load(file)

    response = requests.get(url)
    data = response.json()
    with open(filename, 'w') as file:
        json.dump(data, file)
    return data

def create_connection(db_file):
    """Create a database connection to a SQLite database."""
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Exception as e:
        print(e)
    return None

def create_table(conn, create_table_sql):
    """Create a table from the create_table_sql statement."""
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Exception as e:
        print(e)

def save_data_to_db(df, table_name, conn):
    """Save pandas DataFrame to SQLite database table using an existing connection."""
    df.to_sql(table_name, conn, if_exists='append', index=False)


In [16]:

def process_race_results(data):
    """Process race results data to format suitable for saving to database."""
    races = data['MRData']['RaceTable']['Races']
    if races:
        race = races[0]
        results = race['Results']
        df = pd.DataFrame({
            'season': race['season'],
            'race_name': race['raceName'],
            'round': race['round'],
            'position': [result['position'] for result in results],
            'driver_id': [result['Driver']['driverId'] for result in results],
            'constructor_id': [result['Constructor']['constructorId'] for result in results],
            'laps': [result['laps'] for result in results],
            'time': [result.get('Time', {}).get('time', 'N/A') for result in results],
            'points': [result['points'] for result in results]
        })
        return df
    return pd.DataFrame()

def process_qualifying_results(data):
    """Process qualifying results data."""
    races = data['MRData']['RaceTable']['Races']
    if races:
        qualifyings = races[0]['QualifyingResults']
        df = pd.DataFrame({
            'season': races[0]['season'],
            'race_name': races[0]['raceName'],
            'round': races[0]['round'],
            'position': [qualifying['position'] for qualifying in qualifyings],
            'driver_id': [qualifying['Driver']['driverId'] for qualifying in qualifyings],
            'constructor_id': [qualifying['Constructor']['constructorId'] for qualifying in qualifyings],
            'q1': [qualifying.get('Q1', 'N/A') for qualifying in qualifyings],
            'q2': [qualifying.get('Q2', 'N/A') for qualifying in qualifyings],
            'q3': [qualifying.get('Q3', 'N/A') for qualifying in qualifyings]
        })
        return df
    return pd.DataFrame()

def process_pit_stops(data):
    """Process pit stop data."""
    races = data['MRData']['RaceTable']['Races']
    if races:
        pit_stops = races[0].get('PitStops', [])
        df = pd.DataFrame({
            'season': races[0]['season'],
            'round': races[0]['round'],
            'race_name': races[0]['raceName'],
            'driver_id': [pit['driverId'] for pit in pit_stops],
            'stop': [pit['stop'] for pit in pit_stops],
            'lap': [pit['lap'] for pit in pit_stops],
            'time': [pit['time'] for pit in pit_stops],
            'duration': [pit['duration'] for pit in pit_stops]
        })
        return df
    return pd.DataFrame()

def process_driver_standings(data):
    """Process driver standings data."""
    standings = data['MRData']['StandingsTable']['StandingsLists']
    standings1 = data['MRData']['StandingsTable']['StandingsLists']
    if standings:
        standings = standings[0]['DriverStandings']
        df = pd.DataFrame({
            'season': standings1[0]['season'],
            'round': standings1[0]['round'],
            'driver_id': [standing['Driver']['driverId'] for standing in standings],
            'position': [standing['position'] for standing in standings],
            'points': [standing['points'] for standing in standings],
            'wins': [standing['wins'] for standing in standings]
        })
        return df
    return pd.DataFrame()

def process_constructor_standings(data):
    """Process constructor standings data."""
    standings = data['MRData']['StandingsTable']['StandingsLists']
    standings1 = data['MRData']['StandingsTable']['StandingsLists']
    if standings:
        standings = standings[0]['ConstructorStandings']
        df = pd.DataFrame({
            'season': standings1[0]['season'],
            'round': standings1[0]['round'],
            'constructor_id': [standing['Constructor']['constructorId'] for standing in standings],
            'position': [standing['position'] for standing in standings],
            'points': [standing['points'] for standing in standings],
            'wins': [standing['wins'] for standing in standings]
        })
        return df
    return pd.DataFrame()

def process_fastest_laps(data):
    """Process fastest laps data."""
    races = data['MRData']['RaceTable']['Races']
    if races:
        results = races[0]['Results']
        df = pd.DataFrame({
            'season': races[0]['season'],
            'round': races[0]['round'],
            'race_name': races[0]['raceName'],
            'driver_id': [result['Driver']['driverId'] for result in results],
            'lap': [result.get('FastestLap', {}).get('lap', 'N/A') for result in results],
            'time': [result.get('FastestLap', {}).get('Time', {}).get('time', 'N/A') for result in results],
            'speed': [result.get('FastestLap', {}).get('AverageSpeed', {}).get('speed', 'N/A') for result in results]
        })
        return df
    return pd.DataFrame()


In [4]:
def fetch_all_seasons():
    """Fetch and return a list of all F1 seasons available from the Ergast API."""
    url = "http://ergast.com/api/f1/seasons.json?limit=100"  # Adjust limit if necessary
    response = requests.get(url)
    seasons = []

    if response.status_code == 200:
        data = response.json()
        for item in data['MRData']['SeasonTable']['Seasons']:
            seasons.append(item['season'])
    else:
        print(f"Failed to fetch seasons data: Status code {response.status_code}")
        return None

    return seasons

def fetch_season_rounds(season):
    """Fetch and return the number of rounds for a specific F1 season."""
    url = f"http://ergast.com/api/f1/{season}.json"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        rounds = [race['round'] for race in data['MRData']['RaceTable']['Races']]
        return rounds
    else:
        print(f"Failed to fetch data for season {season}: {response.status_code}")
        return []

def fetch_race_schedule_for_all_seasons(seasons):
    """Fetch race schedules for a list of seasons and return a comprehensive DataFrame."""
    all_races = []

    for year in seasons:
        year_url = f"http://ergast.com/api/f1/{year}.json"
        response = requests.get(year_url)
        if response.status_code == 200:
            data = response.json()
            races = data['MRData']['RaceTable']['Races']
            for race in races:
                race_info = {
                    'season': year,
                    'round': race['round'],
                    'race_name': race['raceName'],
                    'circuit_id': race['Circuit']['circuitId'],
                    'circuit_name': race['Circuit']['circuitName'],
                    'country': race['Circuit']['Location']['country'],
                    'city': race['Circuit']['Location']['locality'],
                    'date': race['date'],
                    'time': race.get('time', 'N/A')  # Include time if available
                }
                all_races.append(race_info)
        else:
            print(f"Failed to fetch race schedule for {year}: Status code {response.status_code}")

    return pd.DataFrame(all_races)

# First, fetch all available seasons
seasons = fetch_all_seasons()

# Check if the seasons list is not empty and proceed
if seasons:
    # Now, fetch race schedules for all these seasons
    all_seasons_race_data = fetch_race_schedule_for_all_seasons(seasons)
else:
    print("No seasons data available or failed to fetch seasons.")


In [12]:
database = "f1_data.db"
conn = create_connection(database)
sql_create_tables = {
        'race_results': """
            CREATE TABLE IF NOT EXISTS race_results (
                season integer,
                race_name text,
                round integer,
                position integer,
                driver_id text,
                constructor_id text,
                laps integer,
                time text,
                points real
            );""",
        'driver_standings': """
            CREATE TABLE IF NOT EXISTS driver_standings (
                season integer,
                round integer,
                driver_id text,
                position integer,
                points real,
                wins integer
            );""",
        'constructor_standings': """
            CREATE TABLE IF NOT EXISTS constructor_standings ( 
                season integer,
                round integer,
                constructor_id text,
                position integer,
                points real,
                wins integer
            );""",
        'qualifying_results': """
            CREATE TABLE IF NOT EXISTS qualifying_results (
                season integer,
                race_name text,
                round integer,
                position integer,
                driver_id text,
                constructor_id text,
                q1 text,
                q2 text,
                q3 text
            );""",
        'pit_stops': """
            CREATE TABLE IF NOT EXISTS pit_stops (
                season integer,
                round integer,
                race_name text,
                driver_id text,
                stop integer,
                lap integer,
                time text,
                duration text
            );""",
        'race_schedule': """
            CREATE TABLE IF NOT EXISTS race_schedule (
                season INTEGER,
                round INTEGER,
                race_name TEXT,
                date TEXT
                circuit_id INTEGER
            );"""
    }


In [41]:
def main():
    conn = create_connection(database)
    for table, sql in sql_create_tables.items():
            create_table(conn, sql)

if seasons:
    for season in range(2024,2026):
        rounds = fetch_season_rounds(season)  # Fetch the valid rounds for each season
        for round in rounds:
            race_results_url = f'http://ergast.com/api/f1/{season}/{round}/results.json'
            race_result_data = fetch_data(race_results_url, season, round, 'race_results')
            if race_result_data:
                df_race_results = process_race_results(race_result_data)
                save_data_to_db(df_race_results, 'race_results', conn)  # Assuming conn is your active db connection

            # Fetch and process driver standings
            driver_standings_url = f'http://ergast.com/api/f1/{season}/{round}/driverStandings.json'
            driver_standings_data = fetch_data(driver_standings_url, season, round, 'driver_standings')
            if driver_standings_data:
                df_driver_standings = process_driver_standings(driver_standings_data)
                save_data_to_db(df_driver_standings, 'driver_standings', conn)

            # Fetch and process constructor standings
            constructor_standings_url = f'http://ergast.com/api/f1/{season}/{round}/constructorStandings.json'
            constructor_standings_data = fetch_data(constructor_standings_url, season, round, 'constructor_standings')
            if constructor_standings_data:
                df_constructor_standings = process_constructor_standings(constructor_standings_data)
                save_data_to_db(df_constructor_standings, 'constructor_standings', conn)

            # Fetch and process qualifying results
            qualifying_results_url = f'http://ergast.com/api/f1/{season}/{round}/qualifying.json'
            qualifying_result_data = fetch_data(qualifying_results_url, season, round, 'qualifying_results')
            if qualifying_result_data:
                df_qualifying_results = process_qualifying_results(qualifying_result_data)
                save_data_to_db(df_qualifying_results, 'qualifying_results', conn)

            # Fetch and process pit stops
            pit_stops_url = f'http://ergast.com/api/f1/{season}/{round}/pitstops.json'
            pit_stop_data = fetch_data(pit_stops_url, season, round, 'pit_stops')
            if pit_stop_data:
                df_pit_stops = process_pit_stops(pit_stop_data)
                save_data_to_db(df_pit_stops, 'pit_stops', conn)

        print(f"Data saved for season {season}")
else:
    print("No seasons data available or failed to fetch seasons.")
        
    if conn:
        conn.close()

if __name__ == "__main__":
    main()

Data saved for season 2024
Data saved for season 2025


In [27]:
save_data_to_db(all_seasons_race_data,'race_schedule',conn)

In [47]:
query = "SELECT * FROM race_results"
df = pd.read_sql_query(query, conn)
df.to_csv('raceresults.csv')

query = "SELECT * FROM qualifying_results"
df = pd.read_sql_query(query, conn)
df.to_csv('qualifyingresults.csv')

query = "SELECT * FROM constructor_standings"
df = pd.read_sql_query(query, conn)
df.to_csv('constructorstandings.csv')

query = "SELECT * FROM driver_standings"
df = pd.read_sql_query(query, conn)
df.to_csv('driverstandings.csv')

query = "SELECT * FROM race_schedule"
df = pd.read_sql_query(query, conn)
df.to_csv('raceschedule.csv')
